In [ ]:
## LIST OF PACKAGES NEEDED

# %pip install pyirt
import irt 
import csv
from pyirt
import warnings
import numpy as np 
import pandas as pd 
import seaborn as sns
import scipy.stats as sp
from sklearn import mixture
import scipy.stats as stats
from kneed import KneeLocator
import scipy.special as special
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
from sklearn.cluster import KMeans
import scipy.integrate as integrate
import matplotlib.gridspec as gridspec
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix 

In [ ]:
## Settings

plt.rc('text', usetex=False) 
plt.rc('font', family='serif') 
fontsize_title = 21 
fontsize_axis = 17 
fontsize_ticks = 13
darkblue = "#2166ac"
lightblue = "#4393c3"
red = "#ff6961"
mediumblue = "cornflowerblue"
palered = "palevioletred"
green = "mediumseagreen"

In [ ]:
class SeabornFig2Grid():
    def __init__(self, seaborngrid, fig, subplot_spec):
        self.fig = fig
        self.sg = seaborngrid
        self.subplot = subplot_spec
        if isinstance(self.sg, sns.axisgrid.FacetGrid) or \
                    isinstance(self.sg, sns.axisgrid.PairGrid):
            self._movegrid()
        elif isinstance(self.sg, sns.axisgrid.JointGrid):
            self._movejointgrid()
        self._finalize()
        
    def _movegrid(self):
        """ Move PairGrid or Facetgrid """
        self._resize()
        n = self.sg.axes.shape[0]
        m = self.sg.axes.shape[1]
        self.subgrid = gridspec.GridSpecFromSubplotSpec(n,m, subplot_spec=self.subplot) 
        for i in range(n):
            for j in range(m):
                self._moveaxes(self.sg.axes[i,j], self.subgrid[i,j])

    def _movejointgrid(self):
        """ Move Jointgrid """
        h= self.sg.ax_joint.get_position().height
        h2= self.sg.ax_marg_x.get_position().height
        r = int(np.round(h/h2))
        self._resize()
        self.subgrid = gridspec.GridSpecFromSubplotSpec(r+1,r+1, subplot_spec=self.subplot) 
        self._moveaxes(self.sg.ax_joint, self.subgrid[1:, :-1]) 
        self._moveaxes(self.sg.ax_marg_x, self.subgrid[0, :-1]) 
        self._moveaxes(self.sg.ax_marg_y, self.subgrid[1:, -1])
        
    def _moveaxes(self, ax, gs): #https://stackoverflow.com/a/46906599/4124317 ax.remove()
        ax.figure=self.fig
        self.fig.axes.append(ax) 
        self.fig.add_axes(ax)
        ax._subplotspec = gs 
        ax.set_position(gs.get_position(self.fig)) 
        ax.set_subplotspec(gs)

    def _finalize(self):
        plt.close(self.sg.fig) 
        self.fig.canvas.mpl_connect("resize_event", self._resize) 
        self.fig.canvas.draw()

    def _resize(self, evt=None): 
        self.sg.fig.set_size_inches(self.fig.get_size_inches())

In [ ]:
dict_labels = {'TT':'Q1','K':'Q2', 'KK':'Q3', 'T':'Q4', 'PE13':'Q5', 'S':'Q6', 'KKK':'Q7', 'PC':'Q8', 'PCPC':'Q9', 'TTT':'Q10', 'PE8':'Q11', 'PE9':'Q12', 'PPE13':'Q13', 'KKKK':'Q14', 'PE12':'Q15', 'PPPE13':'Q16'}

In [ ]:
## LIST OF FUNCTIONS NEEDED THROUGHOUT THE ANALYSIS

def summary(df): 
    F_ = []
    D_ = []
    Q_ = []
    C_ = []
    F_1 = []
    F_2 = []
    E_ = []
    for name, group in df.groupby('ExpNum'):
        for name2, group2 in group.groupby('CodeQuestion'):
            fd, dd = np.mean(group2.Answer*group2.Weight), np.mean(group2.Answer)
            question, code = list(set(list(group2.CodeQuestion)))[0], list(set(list(group2.CodeExclu)))[0] 
            expn = name
            F_.append(fd)
            D_.append(dd)
            Q_.append(question)
            C_.append(code)
            E_.append(expn)
            if fd >= dd:
                F_1.append(fd)
                F_2.append(np.nan) 
            else:
                F_2.append(fd)
                F_1.append(np.nan)
    data_summary = pd.DataFrame(list(zip(F_, D_, Q_,C_, F_1, F_2, E_)), columns = ['LD', 'DD', 'CodeQuestion', 'CodeExclu', 'LDUP', 'LDDOWN', 'ExpNum'])
    return D_, F_, data_summary


def ProbaDel(df): 
    del_proba = []
    for i, r in df.iterrows(): 
        if 'n' in str(r.DelID): 
            del_proba.append(0)
        else: del_proba.append(1)
    df['ProbaDel'] = del_proba 
    return df


def plot_occ(df):
    perc_del = list(df.groupby(['CodeExclu'], as_index = False).mean().ProbaDel)
    dd_v_down = list(df.groupby(['CodeExclu'], as_index = False).std().ProbaDel)
    code = list(df.groupby(['CodeExclu'], as_index = False).mean().CodeExclu)
    ni = list(df.groupby(['CodeExclu'], as_index = False).count().ID)
    code_plot = [dict_labels[k] for k in code]
    perc_del, code, dd_v_down, ni, code_plot = zip(*sorted(zip(perc_del, code, dd_v_down, ni, code_plot)))
    plt.scatter(code_plot, perc_del, color=darkblue) 
    plt.xticks(rotation=90, fontsize=18)
    plt.yticks(rotation=0, fontsize=18)
    plt.title('Frequency of Delegation Per Category', fontsize=21, pad=20) 
    plt.tick_params(which='minor', bottom=False)
    for x in range (len(code)):
        top = perc_del[x] + sp.t.ppf(0.975, ni[x]-1)*dd_v_down[x]/np.sqrt(ni[x]-1) 
        bottom = perc_del[x] - sp.t.ppf(0.975, ni[x]-1)*dd_v_down[x]/np.sqrt(ni[x]-1) 
        plt.plot([code_plot[x], code_plot[x]], [top, bottom], color=darkblue)


def plot_performance_across(data_summary, s):
    fig = plt.figure(figsize=(8, 5))
    plt.suptitle('Direct Democracy vs. Liquid Democracy', fontsize=fontsize_title) 
    ax = fig.add_subplot(121)
    didi = data_summary.groupby(['CodeExclu']).mean().sort_values('DD') 
    print(didi)
    fd_agg = list(didi.LD)
    dd_agg = list(didi.DD)
    codes_agg = [k for k in didi.LD.index]
    cp=0
    ajk=[]
    if s == 'e_f':
        codes_agg = [str(i) for i in range (len(codes_agg))] 
    if s == 'q_f':
        for k in codes_agg:
            if 'P' in k and k!= 'PC':
                ajk.append('P' + str(cp))
                cp+=1 
            else:
                ajk.append(k)
        codes_agg = ajk
    ax.scatter(codes_agg, dd_agg, marker='o', color='cornflowerblue', label='Direct')
    ax.scatter(codes_agg, fd_agg, marker='^', color='palevioletred', label='Liquid')

    ax.set_ylim(0.5, 1)
    ax.set_title('Per Category', fontsize=fontsize_title, pad=20) 
    ax.tick_params(which='minor', axis='x', labelsize=fontsize_axis, bottom=False) 
    ax.tick_params(which='minor', axis='y', labelsize=fontsize_axis, bottom=False) 
    ax.set_xlabel('Category ID', fontsize=fontsize_axis)
    ax.set_ylabel('Correct Answer Rate', fontsize=fontsize_axis) 
    ax.set_ylim(min(min(dd_agg),min(fd_agg))-0.05,max(max(dd_agg),max(fd_agg))+0.05)

    ax = fig.add_subplot(122)
    data_summary = data_summary.loc[data_summary.CodeExclu != 'SR']
    didi = data_summary.groupby(['ExpNum']).mean().sort_values('DD')
    fd_agg = list(didi.LD)
    dd_agg = list(didi.DD)
    codes_agg = [k for k in didi.LD.index]
    codes_agg = ['E'+ str(i) for i in range (len(codes_agg))]
    ax.scatter(codes_agg, dd_agg, marker='o', color='cornflowerblue', label='Direct') 
    ax.scatter(codes_agg, fd_agg, marker='^', color='palevioletred', label='Liquid') 
    ax.set_title('Per Experiment', fontsize=fontsize_title, pad=20) 
    ax.tick_params(which='minor', axis='x', labelsize=fontsize_axis, bottom=False) 
    ax.tick_params(which='minor', axis='y', labelsize=fontsize_axis, bottom=False) 
    ax.set_xlabel('Experiment Number', fontsize=fontsize_axis) 
    ax.set_ylim(min(min(dd_agg),min(fd_agg))-0.05,max(max(dd_agg),max(fd_agg))+0.05) 
    plt.legend(loc='lower right')
    plt.subplots_adjust(right=0.9)
    fig.tight_layout()
    plt.savefig('overall_batch2' + s + '.pdf',bbox_inches='tight')


def double_std(array): 
      return np.std(array) * 2


def ci(array):
    n = len(array)
    return sp.t.ppf(0.975, n-1)*sp.sem(array)*np.sqrt(n/(n-1))

In [ ]:
df = pd.read_csv('data.csv')

# Data Processing

This section processes the data

In [ ]:
focus_on = 'B2'
#"B2" for main study, "B1" for pre-study

In [ ]:
## HIGH LEVEL STATISTICS ON AVERAGE PERFORMANCE

if focus_on == 'B1':
    exp_filter = df.loc[df.ExpNum.isin(['E1','E2','E3','E4','E5','E6'])]
else:
    exp_filter = df.loc[df.ExpNum.isin(['E8','E9','E10','E11','E12','E13'])]
    
print(- np.mean(df.Answer) + np.mean(df.Answer*df.Weight))
print(- np.mean(exp_filter.Answer) + np.mean(exp_filter.Answer*exp_filter.Weight))

In [ ]:
if focus_on == 'B1':
    for k in ['E1','E2','E3','E4','E5','E6']:
        exp_filter_ = exp_filter.loc[exp_filter.ExpNum==k]
        print(np.mean(exp_filter_.Answer), np.mean(exp_filter_.Answer*exp_filter_.Weight))
    else:
        for k in ['E8','E9','E10','E11','E12','E13']:
            exp_filter_ = exp_filter.loc[exp_filter.ExpNum==k]
            print(np.mean(exp_filter_.Answer), np.mean(exp_filter_.Answer*exp_filter_.Weight))  

In [ ]:
## behavior has the average per user per category of metrics such as expertise and information constant across users (such as gender)
## behavior_agg is aggregate per use, so that it accounts for probability of delegating

In [ ]:
dct= {
    'number': 'mean',
    'object': lambda col: col.mode() if col.nunique() == 1 else np.nan,
}
groupby_cols = ['ExpNum', 'ID']
dct = {k: v for i in [{col: agg for col in exp_filter.select_dtypes(tp).columns.difference(groupby_cols)} for tp, agg in dct.items()] for k, v in i.items()}
behavior_agg = exp_filter.groupby(groupby_cols).agg(**{k: (k, v) for k, v in dct.items()}).reset_index()

dct= {
    'number': 'mean',
    'object': lambda col: col.mode() if col.nunique() == 1 else np.nan,
}
groupby_cols = ['ExpNum', 'ID', 'CodeExclu']
dct = {k: v for i in [{col: agg for col in exp_filter.select_dtypes(tp).columns.difference(groupby_cols)} for tp, agg in dct.items()] for k, v in i.items()}
behavior = exp_filter.groupby(groupby_cols).agg(**{k: (k, v) for k, v in dct.items()}).reset_index()

# Statistics

This section get high-level statistics of the data

In [ ]:
if focus_on == 'B1':
    exp_ = ['E1','E2','E3','E4','E5','E6']
else:
    exp_ = ["E8", "E9", "E10", "E11", "E12", "E13"]
tot = 0
for k in exp_:
    print("Number of participants in ", k, " ", str(len(list(set(df.loc[df.ExpNum == k].ID)))))
    tot += len(list(set(df.loc[df.ExpNum == k].ID)))
print("Number of total participants: ", tot)

## Weight Distribution

In [ ]:
w = list(exp_filter.groupby('Weight').count().index)
c = list(exp_filter.groupby('Weight').count().ID) 
plt.figure(figsize=(5, 4))
plt.xticks(rotation=0, fontsize=fontsize_axis, label='Weight') 
plt.yticks(rotation=0, fontsize=fontsize_axis, label='Occurences') 
plt.title('Hitogram of Weights', fontsize=fontsize_title, pad=20) 
plt.tick_params(which='minor', bottom=False)
plt.xlabel('Weights', fontsize=fontsize_axis)
plt.ylabel('Occurences', fontsize=fontsize_axis)
plt.scatter(w, c, color=darkblue)
plt.tight_layout()

In [ ]:
if focus_on == 'B1':
    dict_size = {"E1": 11, "E2": 12, "E3": 32, "E4": 14, "E5": 17, "E6": 15}
else:
    dict_size = {"E8": 14, "E9": 22, "E10": 19, "E11": 27, "E12": 36, "E13": 50}
size, max_w = [], []
avgsize, avgmax_w = [], []
for name, group in exp_filter.groupby('ExpNum'):
    for k in list(group.groupby('CodeExclu').max().Weight): 
        size.append(dict_size[name])
        max_w.append(k)
    avgsize.append(dict_size[name])
    avgmax_w.append(np.median(list(group.groupby('CodeExclu').max().Weight)))

plt.figure(figsize=(5, 4))
plt.xticks(rotation=0, fontsize=fontsize_axis, label='N') 
plt.yticks(rotation=0, fontsize=fontsize_axis, label='Max Weight') 
plt.title('Maximum Weight Per Task', fontsize=fontsize_title, pad=20) 
plt.tick_params(which='minor', bottom=False)
plt.xlabel('Experiment Size ($N_e$)', fontsize=fontsize_axis)
plt.ylabel('Maximum Weight', fontsize=fontsize_axis)
plt.scatter(size, max_w, color=darkblue)
plt.scatter(avgsize, avgmax_w, color=red)
plt.tight_layout()

In [ ]:
data = []
e = []
e_label = []
for name, group in exp_filter.groupby('ExpNum'):
    e.append(dict_size[name])
    e_label.append(str(dict_size[name]) + "\n (E" + str(int(name.split('E')[1])-7) + ")") 
    max_w = []
    for k in list(group.groupby('CodeExclu').max().Weight):
            max_w.append(k)
            print(k > dict_size[name]/2)
    data.append(max_w)

medians = [np.median(d) for d in data]

e, data, e_label = zip(*sorted(zip(e, data, e_label), key=lambda x: np.median(x[0])))
c = mediumblue

if focus_on == 'B1':
    fig, ax = plt.subplots(figsize=(10, 5)) 
    ax.boxplot(data, notch=False, patch_artist=True,
                    boxprops=dict(facecolor=c, color=c),
                    capprops=dict(color=c),
                    whiskerprops=dict(color=c),
                    flierprops=dict(color=c, markeredgecolor=c), medianprops=dict(color=red), positions=e, widths=0.5)
    ax.set_xlabel('Group Size', fontsize=fontsize_axis)
    ax.set_ylabel('Maximum Weight', fontsize=fontsize_axis)
    ax.set_title('Maximum Weight Box Plot', fontsize=fontsize_title)
    ax.set_xticklabels(e_label)
    ax.set_xlim([9, 35])
    ax.plot([9, 35], [9/2, 35/2], color = green, label = '$N_e/2$')
    ax.legend()
    plt.savefig('weight_pre.pdf')
else:
    fig, ax = plt.subplots(figsize=(10, 5)) 
    ax.boxplot(data, notch=False, patch_artist=True,
                    boxprops=dict(facecolor=c, color=c),
                    capprops=dict(color=c),
                    whiskerprops=dict(color=c),
                    flierprops=dict(color=c, markeredgecolor=c), medianprops=dict(color=red), positions=e, widths=3)
    ax.set_xlabel('Group Size', fontsize=fontsize_axis)
    ax.set_ylabel('Maximum Weight', fontsize=fontsize_axis)
    ax.set_title('Maximum Weight Box Plot', fontsize=fontsize_title)
    ax.set_xticklabels(e_label)
    ax.set_xlim([10, 55])
    ax.plot([10, 55], [10/2, 55/2], color = green, label = '$N_e/2$')
    ax.legend()
    plt.savefig('weight.pdf')

## Delegation Frequency

In [ ]:
c = mediumblue
delegation_per_gender = behavior_agg.groupby("gender").agg([np.mean, double_std, ci])["ProbaDel"] 
delegation_per_gender = delegation_per_gender.reset_index()
delegation_per_gender.gender = delegation_per_gender.gender.str.replace("prefer to self-describe", "self-describe") 
delegation_per_gender.plot(kind = "scatter", x = "gender", y = "mean", legend = False, title = "", yerr = "ci", color = c, s=50)
plt.tick_params(axis='x', labelsize=fontsize_ticks, rotation=0)
plt.ylabel('Delegation Frquency', fontsize=fontsize_axis)
plt.xlabel('Gender', fontsize=fontsize_axis)
plt.tick_params(axis='y', labelsize=fontsize_ticks)
plt.title('Delegation Frquency Per Gender', fontsize=fontsize_title)

In [ ]:
behavior_agg[behavior_agg['gender'].notna()].to_csv('anova.csv')
plot_occ(behavior)

# Estimating $q$ and $\varphi$

This section computes the expertise accounting for individual and questions heterogeneity, discusses the bucketing strategies and how to compute $\varphi$

In [ ]:
exp_filter['unique_id'] = exp_filter['ExpNum'] + '_' + exp_filter['ID'].astype(str)
exp_filter['unique_id_q'] = exp_filter['ExpNum'] + '_' + exp_filter['ID'].astype(str) + '_' + exp_filter['CodeExclu']
expertise = exp_filter.groupby(['ID', 'ExpNum', 'CodeExclu', "unique_id", "unique_id_q"]).mean().AverageExpertise.reset_index()

questions_score = {}
expertise['IRTExpertise'] =  pd.Series([0] * len(expertise))

for name, group in exp_filter.groupby(["CodeExclu"]):
    l = list(group[["unique_id_q", "CodeQuestion", "Answer"]].itertuples(index=False, name=None))
    #item_param, user_param = irt(l)
    #item_param, user_param = irt(l, theta_bnds = [0,1], alpha_bnds=[0,1], beta_bnds = [0,1])
    #item_param, user_param = irt(l, in_guess_param = {1:{'c':0.0}, 2:{'c':0.5}, 3:{'c':1}})
    item_param, user_param = irt(l)
    questions_score.update(item_param)
    update_b = lambda x: user_param[x['unique_id_q']] if x['unique_id_q'] in user_param else x['IRTExpertise'] 
    expertise['IRTExpertise'] = expertise.apply(update_b, axis=1)

expertise["Normalized_IRT"]  = expertise["IRTExpertise"].subtract(min(expertise["IRTExpertise"])).divide(max(expertise["IRTExpertise"])-min(expertise["IRTExpertise"]))
expertise["Normalized_IRT_1"]  = expertise["IRTExpertise"].subtract(min(expertise["IRTExpertise"])).divide((max(expertise["IRTExpertise"])-min(expertise["IRTExpertise"]))/20).subtract(10)

np.corrcoef(expertise.AverageExpertise, expertise.Normalized_IRT), np.corrcoef(expertise.AverageExpertise, expertise.Normalized_IRT_1)

In [ ]:
merged_df = pd.merge(behavior, expertise, left_on=['ExpNum', 'ID', 'CodeExclu'], right_on=['ExpNum', 'ID', 'CodeExclu'], how='left')
merged_df_flat = pd.merge(exp_filter, expertise[['IRTExpertise', 'Normalized_IRT', 'Normalized_IRT_1', 'unique_id_q']], on=["unique_id_q"], how='left')
merged_df = merged_df.loc[~(merged_df.CodeExclu == 'S')]

if focus_on == 'B2': 
    merged_df.to_csv('outcome_merged_batch2_lab.csv') 
    merged_df_flat.to_csv('outcome_merged_flat_batch2_lab.csv')
else:
    merged_df.to_csv('outcome_merged_batch1_lab.csv') 
    merged_df_flat.to_csv('outcome_merged_flat_batch1_lab.csv')

In [ ]:
fig, axes = plt.subplots(2,3, figsize=(15,7))
gg_, i_, mini_ = [], [], []
e, e_label = [], []
data = []
dict_all = {}
aks = 0
for name, group in merged_df.groupby(['ExpNum']):
    ax = axes.flatten()[aks]
    ma = []
    for n, gg in group.groupby('CodeExclu'):
        di = {}
        g = gg.loc[gg.Weight!=0]
        l = sorted(g.Weight, reverse = True) 
        s=0
        i=1
        mini = 100
        for k in l:
            s += k
            dict_all[round(i,2)] = round(s/len(gg), 2) 
            di[i] = round(s/len(gg), 2)
            if s/len(gg) > 0.5:
                mini = min(mini,i)
            i += 1
        print(len(gg), i, mini)
        gg_.append(len(gg))
        i_.append(i)
        mini_.append(mini)
        ma.append(mini)
        ax.scatter(list(di.keys()),list(di.values()))
    print(name)
    data.append(ma)
    e.append(len(gg))
    e_label.append(str(len(gg)) + "\n (E" + str(int(name.split('E')[1])-7) + ")") 
    ax.set_xlabel("Number of Gurus", fontsize=16)
    ax.set_title("Experiment " + str(int(name.split('E')[1])-7) + ' ($N_e=$' + str(len(gg)) + ")")
    aks += 1

plt.suptitle('Fraction of Votes (y) Controled by x Gurus', fontsize=fontsize_title)
plt.tight_layout()
plt.savefig('weights_cdf.pdf')

In [ ]:
plt.scatter(mini_, i_, color = mediumblue)
plt.suptitle('Number of Gurus as a Function \n of Minimal Number of Gurus \n that Gathered Half of the Votes', fontsize=fontsize_title)
plt.tight_layout()
plt.savefig('gurus.pdf')

In [ ]:
if focus_on == 'B2': 
    merged_df.groupby('ID').mean()
    dct = {
        'number': 'mean',
        'object': lambda col: col.mode() if col.nunique() == 1 else np.nan,
    }
    groupby_cols = ['unique_id']
    dct = {k: v for i in [{col: agg for col in merged_df.select_dtypes(tp).columns.difference(groupby_cols)} for tp, agg in dct.items()] for k, v in i.items()}
    anova = merged_df.groupby(groupby_cols).agg(**{k: (k, v) for k, v in dct.items()}).reset_index() 
    anova[anova['gender'].notna()].to_csv('anova.csv')
    anova.to_csv('anova.csv')

behavior.to_csv('behavior.csv')
behavior = pd.read_csv('behavior.csv')

In [ ]:
fig = plt.figure(figsize=(6,3))
ax = fig.add_subplot(111)
behavior.AverageExpertise.hist(color=mediumblue, label='Naive Expertise: $p^{naive}_i$', bins=15, density = True) 
expertise.Normalized_IRT.hist(color='red', label='IRT Expertise: $p_i$', bins=15, alpha=0.5, density = True)
ax.set_title('Distribution of Expertise', fontsize=fontsize_title, pad=20) 
ax.tick_params(which='minor', axis='x', labelsize=fontsize_axis, bottom=False) 
ax.tick_params(which='minor', axis='y', labelsize=fontsize_axis, bottom=False) 
ax.set_xlabel('Expertise', fontsize=fontsize_axis)
ax.set_ylabel('Occurences', fontsize=fontsize_axis) 
ax.legend()
ax.grid(False)
fig.tight_layout()
plt.subplots_adjust(left=0.1,
                    bottom=0.1,
                    right=0.9,
                    top=0.9,
                    wspace=0.2,
                    hspace=0.9)
if focus_on == 'B1': 
    plt.savefig('expertise_distribution_pre.pdf',bbox_inches='tight')
else: 
    plt.savefig('expertise_distribution_p.pdf',bbox_inches='tight')

## Bucketing

We then bucket the computed expertise for the $\varphi$ analysis

### Gaussian Mixture 

(One of the bucketing methods)

In [ ]:
X = np.array(list(merged_df.Normalized_IRT)).reshape(-1, 1) 

# Define a range of n_components values to try
n_components_range = range(1, 11) 

def log_likelihood(estimator, X):
    return np.sum(estimator.score_samples(X))

# Perform cross-validation
cv_scores = []
for n_components in n_components_range:
    gmm = mixture.GaussianMixture(n_components=n_components,n_init=100,max_iter=50000, covariance_type='full', init_params='random')
    scores = cross_val_score(gmm, X, cv=5, scoring=log_likelihood)
    cv_scores.append(np.mean(scores))

# Find the optimal number of components with the highest cross-validation score
optimal_n_components = n_components_range[np.argmax(cv_scores)]

# Train the final GMM using the optimal number of components
final_gmm = mixture.GaussianMixture(n_components=optimal_n_components)
final_gmm.fit(X)

# Evaluate the performance of the final GMM
final_score = final_gmm.score(X)

# Print the optimal number of components and the final score
print("Optimal number of components:", optimal_n_components)
print("Final GMM score:", final_score)

In [ ]:
# Obtain the mean values for each class
class_means = final_gmm.means_.flatten()

# Sort the mean values and get the corresponding indices
sorted_indices = np.argsort(class_means)

# Create a mapping dictionary for new labels
new_labels = {old_label: new_label for old_label, new_label in enumerate(sorted_indices)}

# Update the labels of the GMM model
gmm_labels = final_gmm.predict(np.array(list(merged_df.Normalized_IRT)).reshape(-1, 1)) 
new_gmm_labels = [new_labels[label] for label in gmm_labels]

In [ ]:
class_means = final_gmm.means_.flatten() 
dict_m = {}
for k in range (len(class_means)):
    dict_m[k] = class_means[k]

In [ ]:
merged_df["Labels_GMM"] = final_gmm.predict(np.array(list(merged_df.Normalized_IRT)).reshape(-1, 1))
merged_df["Labels_GMM"] = merged_df["Labels_GMM"].map(dict_m)


In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
for label, df in merged_df.groupby('Labels_GMM'):
    df.Normalized_IRT.plot(kind="kde", ax=ax, label=label)
plt.legend()

### K-means clustering

(Another of the bucketing strategies)

In [ ]:
def find_closest(x):
    return min(l, key=lambda y: abs(y - x)**2)

In [ ]:
kmeans_kwargs = {"init": "k-means++","n_init": 10,"max_iter": 10000,"random_state": 42,}
sse = []
for k in range(1, 25):
    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(X)
    sse.append(kmeans.inertia_)
kl = KneeLocator(range(1, 25), sse, curve="convex", direction="decreasing")
kl.elbow

In [ ]:
kmeans = KMeans(init="k-means++", n_clusters=kl.elbow,n_init=10, max_iter=300, random_state=42) 
kmeans.fit(X)
l = kmeans.cluster_centers_
l = [k[0] for k in l]

In [ ]:
merged_df['Labels_Kmeans'] = merged_df['Normalized_IRT'].apply(find_closest)

In [ ]:
fig = plt.figure(figsize=(12,12))

ax = fig.add_subplot(321)
ax.plot(range(1, 25), sse, color=c)
ax.set_title('Sum of Squared Distances \n of Samples to Closest Cluster Center \n per Cluster Size', fontsize=fontsize_title, pad=20)
ax.tick_params(which='minor', axis='x', labelsize=fontsize_axis, bottom=False)
ax.tick_params(which='minor', axis='y', labelsize=fontsize_axis, bottom=False)
ax.set_xlabel('Expertise', fontsize=fontsize_axis)
ax.set_ylabel('Density', fontsize=fontsize_axis)

ax = fig.add_subplot(322)
for label, df in merged_df.groupby('Labels_Kmeans'):
    df.Normalized_IRT.plot(kind="hist", ax=ax, label=label)
ax.set_title('Empirical Distribution \n of Expertise with \n 4 K-Mean Clusters', fontsize=fontsize_title, pad=20) 
ax.tick_params(which='minor', axis='x', labelsize=fontsize_axis, bottom=False)
ax.tick_params(which='minor', axis='y', labelsize=fontsize_axis, bottom=False)
ax.set_xlabel('Expertise', fontsize=fontsize_axis)
ax.set_ylabel('Density', fontsize=fontsize_axis)
ax.set_xlim(0, 1)

fig.tight_layout()
plt.subplots_adjust(left=0.1,
                    bottom=0.1,
                    right=0.9,
                    top=0.9,
                    wspace=0.2,
                    hspace=0.9)

if focus_on == 'B1': 
    plt.savefig('bucketing1.pdf',bbox_inches='tight')
else: 
    plt.savefig('bucketing2.pdf',bbox_inches='tight')

## Compute Phi

We use the maximum likelihood estimator here

In [ ]:
# Compute the number of times where someone with Pi delegates to someone with Pj, stored in Z_MAT as a B*B matrix
# (B is the number of buckets)
def compute_z(df, expnum, code):
    d13 = df.loc[(df.ExpNum == expnum) & (df.CodeExclu == code)]
    sdf = d13.loc[d13.ProbaDel == 1]

    PI = d13.groupby('Buckets').count().ID.reset_index()
    PI_dict = dict(zip(list(PI.Buckets), list(PI.ID)))
    PI_dict = {k: v for k, v in PI_dict.items() if v > 0}

    s_PI = sdf.groupby('Buckets').count().ID.reset_index()
    s_PI_dict = dict(zip(list(s_PI.Buckets), list(s_PI.ID)))
    s_PI_dict = {k: v for k, v in s_PI_dict.items() if v > 0}

    v_exp_id, v_exp_del = [], []
    v_conf_id, v_conf_del = [], []

    codeq, idn = [], []
    for i, r in d13.iterrows():
        if r.Weight == 0:
            v_exp_id.append(r.Buckets)
            del_exp = \
            list(d13.loc[(d13.ID == r.DelID) & (d13.CodeExclu == r.CodeExclu) & (d13.ExpNum == r.ExpNum)].Buckets)[0]
            v_exp_del.append(del_exp)
            idn.append(r.ID)
        if r.ID == r.GuruID and r.ProbaDel == 1 and len(d13.loc[d13.ID == r.DelID]):
            # print(expnum, code, 'cycle')
            # print(r)
            # print(d13.loc[d13.ID == r.DelID])
            v_exp_id.append(r.Buckets)
            del_exp = \
            list(d13.loc[(d13.ID == r.DelID) & (d13.CodeExclu == r.CodeExclu) & (d13.ExpNum == r.ExpNum)].Buckets)[0]
            v_exp_del.append(del_exp)
            idn.append(r.ID)

    data_phi13 = pd.DataFrame(list(zip(v_exp_id, v_exp_del, idn)), columns=['I', 'J', 'Count'])

    # print(expnum, code, data_phi13)
    Z = data_phi13.groupby(['I', 'J']).count().reset_index()
    B = sorted(list(set(list(d13.Buckets))))
    Z_MAT = []
    for pi in B:
        Z_j = []
        for pj in B:
            try:
                Z_j.append(list(Z.loc[(Z.I == pi) & (Z.J == pj)].Count)[0])
            except:
                Z_j.append(0)
        Z_MAT.append(Z_j)
    return Z_MAT, PI_dict, s_PI_dict


# Create the left-hand side of the system of linear equations
def computer_M(Z_MAT, map_bucket_to_index, i, PI_dict, B, e, c):
    M_I = []
    Mi = []
    pi_i = PI_dict[i]
    z_ii = Z_MAT[map_bucket_to_index[i]][map_bucket_to_index[i]]
    pi_i = sum(Z_MAT[map_bucket_to_index[i]])
    for l in B:
        if l == i:
            v = (PI_dict[i] - 1) * (z_ii - pi_i)
            Mi.append(v)
        else:
            v = z_ii * PI_dict[l]
            Mi.append(v)
    M_I.append(Mi)
    for j in B:
        if j != i:
            Mj = []
            # print(e, c, Z_MAT, i, j, map_bucket_to_index, PI_dict)
            z_ij = Z_MAT[map_bucket_to_index[i]][map_bucket_to_index[j]]
            pi_j = PI_dict[j]
            for l in B:
                if l == i:
                    v = z_ij * (PI_dict[i] - 1)
                    Mj.append(v)
                if l == j:
                    v = (z_ij - pi_i) * pi_j
                    Mj.append(v)
                if l not in [i, j]:
                    v = z_ij * PI_dict[l]
                    Mj.append(v)
            M_I.append(Mj)
    return M_I


def get_dataframe(df, i):
    df_phi_i = pd.DataFrame(columns=['pj', 'Weight', 'ExpNum', 'CodeExclu'])
    for e in list(set(df.ExpNum)):
        for c in list(set(df.CodeExclu)):
            Z_MAT, PI_dict, s_PI_dict = compute_z(df, e, c)
            # print(e, c)
            map_ = {}
            ki = 0
            for k in list(PI_dict.keys()):
                map_[k] = ki
                ki += 1
            # print(map_, PI_dict, list(PI_dict.keys()), i)
            # print(e, c, i, i in s_PI_dict.keys())
            if i in s_PI_dict.keys() and PI_dict[i] > 1:
                # print(PI_dict[i])
                M_I = computer_M(Z_MAT, map_, i, PI_dict, list(PI_dict.keys()), e, c)
                M_I[0] = [1 for i in range(len(M_I[-1]))]
                A = np.array(M_I)
                y = np.array([1] + [0] * (len(PI_dict) - 1))
                try:
                    x = np.linalg.solve(A, y)
                    # print(list(PI_dict.keys()), x)
                    ee_vec, cc_vec = [e] * len(x), [c] * len(x)
                    df_phi2 = pd.DataFrame(list(zip(list(PI_dict.keys()), x, ee_vec, cc_vec)),
                                           columns=['pj', 'Weight', 'ExpNum', 'CodeExclu'])
                    df_phi_i = df_phi_i.append(df_phi2, ignore_index=True)
                    # print(x)
                    # print(sum(x))
                # print(list(PI_dict.keys()), x)
                except:
                    print(e, c, i)
                    print(A)
                    print('Singular Matrix')
            if i in s_PI_dict.keys() and PI_dict[i] == 1:
                # print(e, c, i, Z_MAT[map_[i]])
                ee_vec, cc_vec = [e] * len(Z_MAT[map_[i]]), [c] * len(Z_MAT[map_[i]])
                df_phi2 = pd.DataFrame(list(zip(list(PI_dict.keys()), Z_MAT[map_[i]], ee_vec, cc_vec)),
                                       columns=['pj', 'Weight', 'ExpNum', 'CodeExclu'])
                df_phi_i = df_phi_i.append(df_phi2, ignore_index=True)
    return df_phi_i

In [ ]:
# Compute the number of times where someone with Pi delegates to someone with Pj, stored in Z_MAT as a B*B matrix
# (B is the number of Labels_GMM)
def compute_z_gmm(df, expnum, code):
    d13 = df.loc[(df.ExpNum == expnum) & (df.CodeExclu == code)]
    sdf = d13.loc[d13.ProbaDel == 1]

    PI = d13.groupby('Labels_GMM').count().ID.reset_index()
    PI_dict = dict(zip(list(PI.Labels_GMM), list(PI.ID)))
    PI_dict = {k: v for k, v in PI_dict.items() if v > 0}

    s_PI = sdf.groupby('Labels_GMM').count().ID.reset_index()
    s_PI_dict = dict(zip(list(s_PI.Labels_GMM), list(s_PI.ID)))
    s_PI_dict = {k: v for k, v in s_PI_dict.items() if v > 0}

    v_exp_id, v_exp_del = [], []
    v_conf_id, v_conf_del = [], []

    codeq, idn = [], []
    for i, r in d13.iterrows():
        if r.Weight == 0:
            v_exp_id.append(r.Labels_GMM)
            del_exp = \
            list(d13.loc[(d13.ID == r.DelID) & (d13.CodeExclu == r.CodeExclu) & (d13.ExpNum == r.ExpNum)].Labels_GMM)[0]
            v_exp_del.append(del_exp)
            idn.append(r.ID)
        if r.ID == r.GuruID and r.ProbaDel == 1 and len(d13.loc[d13.ID == r.DelID]):
            # print(expnum, code, 'cycle')
            # print(r)
            # print(d13.loc[d13.ID == r.DelID])
            v_exp_id.append(r.Labels_GMM)
            del_exp = \
            list(d13.loc[(d13.ID == r.DelID) & (d13.CodeExclu == r.CodeExclu) & (d13.ExpNum == r.ExpNum)].Labels_GMM)[0]
            v_exp_del.append(del_exp)
            idn.append(r.ID)

    data_phi13 = pd.DataFrame(list(zip(v_exp_id, v_exp_del, idn)), columns=['I', 'J', 'Count'])

    # print(expnum, code, data_phi13)
    Z = data_phi13.groupby(['I', 'J']).count().reset_index()
    B = sorted(list(set(list(d13.Labels_GMM))))
    Z_MAT = []
    for pi in B:
        Z_j = []
        for pj in B:
            try:
                Z_j.append(list(Z.loc[(Z.I == pi) & (Z.J == pj)].Count)[0])
            except:
                Z_j.append(0)
        Z_MAT.append(Z_j)
    return Z_MAT, PI_dict, s_PI_dict


# Create the left-hand side of the system of linear equations
def computer_M_gmm(Z_MAT, map_bucket_to_index, i, PI_dict, B, e, c):
    M_I = []
    Mi = []
    pi_i = PI_dict[i]
    z_ii = Z_MAT[map_bucket_to_index[i]][map_bucket_to_index[i]]
    pi_i = sum(Z_MAT[map_bucket_to_index[i]])
    for l in B:
        if l == i:
            v = (PI_dict[i] - 1) * (z_ii - pi_i)
            Mi.append(v)
        else:
            v = z_ii * PI_dict[l]
            Mi.append(v)
    M_I.append(Mi)
    for j in B:
        if j != i:
            Mj = []
            # print(e, c, Z_MAT, i, j, map_bucket_to_index, PI_dict)
            z_ij = Z_MAT[map_bucket_to_index[i]][map_bucket_to_index[j]]
            pi_j = PI_dict[j]
            for l in B:
                if l == i:
                    v = z_ij * (PI_dict[i] - 1)
                    Mj.append(v)
                if l == j:
                    v = (z_ij - pi_i) * pi_j
                    Mj.append(v)
                if l not in [i, j]:
                    v = z_ij * PI_dict[l]
                    Mj.append(v)
            M_I.append(Mj)
    return M_I


def get_dataframe_gmm(df, i):
    df_phi_i = pd.DataFrame(columns=['pj', 'Weight', 'ExpNum', 'CodeExclu'])
    for e in list(set(df.ExpNum)):
        for c in list(set(df.CodeExclu)):
            Z_MAT, PI_dict, s_PI_dict = compute_z_gmm(df, e, c)
            # print(e, c)
            map_ = {}
            ki = 0
            for k in list(PI_dict.keys()):
                map_[k] = ki
                ki += 1
            # print(map_, PI_dict, list(PI_dict.keys()), i)
            # print(e, c, i, i in s_PI_dict.keys())
            if i in s_PI_dict.keys() and PI_dict[i] > 1:
                # print(PI_dict[i])
                M_I = computer_M_gmm(Z_MAT, map_, i, PI_dict, list(PI_dict.keys()), e, c)
                M_I[0] = [1 for i in range(len(M_I[-1]))]
                A = np.array(M_I)
                y = np.array([1] + [0] * (len(PI_dict) - 1))
                try:
                    x = np.linalg.solve(A, y)
                    # print(list(PI_dict.keys()), x)
                    ee_vec, cc_vec = [e] * len(x), [c] * len(x)
                    df_phi2 = pd.DataFrame(list(zip(list(PI_dict.keys()), x, ee_vec, cc_vec)),
                                           columns=['pj', 'Weight', 'ExpNum', 'CodeExclu'])
                    df_phi_i = df_phi_i.append(df_phi2, ignore_index=True)
                    # print(x)
                    # print(sum(x))
                # print(list(PI_dict.keys()), x)
                except:
                    print(e, c, i)
                    print(A)
                    print('Singular Matrix')
            if i in s_PI_dict.keys() and PI_dict[i] == 1:
                # print(e, c, i, Z_MAT[map_[i]])
                ee_vec, cc_vec = [e] * len(Z_MAT[map_[i]]), [c] * len(Z_MAT[map_[i]])
                df_phi2 = pd.DataFrame(list(zip(list(PI_dict.keys()), Z_MAT[map_[i]], ee_vec, cc_vec)),
                                       columns=['pj', 'Weight', 'ExpNum', 'CodeExclu'])
                df_phi_i = df_phi_i.append(df_phi2, ignore_index=True)
    return df_phi_i

In [ ]:
# We check here that the short closed form matches the unormalized solution above

e='E13'
c = 'K'
Z_MAT, PI_dict, s_PI_dict = compute_z_gmm(merged_df, e, c) 
map_ = {}
ki = 0
for k in list(PI_dict.keys()):
    map_[k] = ki
    ki += 1 
phi = []
B = sorted(list(set(list(merged_df.Labels_GMM)))) 
for i in B:
    phi_j = [] 
    for j in B:
        z_ij = Z_MAT[map_[i]][map_[j]] 
        if j == i:
            pi_j = PI_dict[j]-1 
        else:
            pi_j = PI_dict[j]
        phi_j.append(z_ij/pi_j)
    n_p = []
    for k in phi_j:
        n_p.append(k/sum(phi_j))
    phi.append(n_p)

In [ ]:
buckets_num = 8
delta = (max(merged_df.Normalized_IRT) - min(merged_df.Normalized_IRT))/buckets_num
cut_offs = [delta*i for i in range(buckets_num + 1)]
labels = [round((cut_offs[i]+cut_offs[i+1])/2,2) for i in range (buckets_num)]
merged_df['Buckets'] = pd.cut(merged_df.Normalized_IRT, bins = cut_offs, labels=labels, include_lowest=True)

### Equal Cut

In [ ]:
labels = {}
for n, g in merged_df.groupby('Buckets'):
             labels[n] = np.mean(g.Normalized_IRT)

merged_df['Buckets'] = merged_df['Buckets'].map(labels)


### Quantile Cut

In [ ]:
merged_df['Buckets'] = pd.qcut(merged_df.Normalized_IRT, buckets_num) 
labels = []
for n, g in merged_df.groupby('Buckets'):
    labels.append(np.median(g.Normalized_IRT))

merged_df['Buckets'] = pd.qcut(merged_df.Normalized_IRT, buckets_num, labels =labels)

In [ ]:
method = 'Buckets' #'Buckets', "GMM"
merged_df["Labels_GMM"] = merged_df["Labels_Kmeans"]

In [ ]:
if method == 'Buckets':
    B = sorted(list(set(list(merged_df.Buckets))))
    df_phi_tot = pd.DataFrame(columns=['pj', 'Weight'])
    kf = 0
    d_weights_total = pd.DataFrame(columns=['pj', 'Weight', 'ExpNum', 'CodeExclu', 'pi']) 
    for i in B:
        df_phi_i = get_dataframe(merged_df[['ID', 'DelID', 'CodeExclu', 'ExpNum', 'Buckets', 'Weight', 'ProbaDel', 'GuruID']], i)
        df_phi_i["pi"] = [i]*len(df_phi_i)

        # Store the per exp per task (pi, pj) pair
        d_weights_total = d_weights_total.append(df_phi_i, ignore_index=True)
        
        # Take the average weight per pair
        new_d = df_phi_i.groupby('pj').mean().reset_index()
        dd_i = pd.DataFrame(list(zip(list(df_phi_i.pj), list(df_phi_i.Weight))), columns=['pj', 'Weight']) 
        df_phi_tot = df_phi_tot.append(dd_i, ignore_index=True)

if method == 'gmm':
    B = sorted(list(set(list(merged_df.Labels_GMM))))
    df_phi_tot = pd.DataFrame(columns=['pj', 'Weight'])
    kf = 0
    d_weights_total = pd.DataFrame(columns=['pj', 'Weight', 'ExpNum', 'CodeExclu', 'pi']) 
    for i in B:
        df_phi_i = get_dataframe_gmm(merged_df[['ID', 'DelID', 'CodeExclu', 'ExpNum', 'Labels_GMM', 'Weight', 'ProbaDel', 'GuruID']], i)
        df_phi_i["pi"] = [i]*len(df_phi_i)

        # Store the per exp per task (pi, pj) pair
        d_weights_total = d_weights_total.append(df_phi_i, ignore_index=True)

        # Take the average weight per pair
        new_d = df_phi_i.groupby('pj').mean().reset_index()
        dd_i = pd.DataFrame(list(zip(list(df_phi_i.pj), list(df_phi_i.Weight))), columns=['pj', 'Weight']) 
        df_phi_tot = df_phi_tot.append(dd_i, ignore_index=True)

In [ ]:
res = stats.kendalltau(d_weights_total.Weight, d_weights_total.pj)
print(res)
for name, group in d_weights_total.groupby('pi'): 
    print(stats.kendalltau(group.Weight, group.pj))

In [ ]:
tot=0
for i in B:
    filter_ = d_weights_total.loc[d_weights_total.pi == i]
    new_d = filter_.groupby('pj').agg({'Weight': 'mean'}).reset_index()
    print(sum(new_d.Weight))
    new_d.Weight = new_d.Weight.divide(sum(new_d.Weight))
    x = np.array(list(new_d.pj)).reshape((-1, 1))
    y = np.array(list(new_d.Weight))
    model = LinearRegression().fit(x, y)
    r_sq = model.score(x, y)

    print('coefficient of determination:', r_sq)
    print('intercept:', model.intercept_)
    print('slope:', model.coef_)

    tot += model.coef_[0]
    x = np.array(list(filter_.pj)).reshape((-1, 1))
    y = np.array(list(filter_.Weight))
    model = LinearRegression().fit(x, y)
    r_sq = model.score(x, y)

    print('coefficient of determination:', r_sq)
    print('intercept:', model.intercept_)
    print('slope:', model.coef_)
    print()
tot

In [ ]:
fig, axes = plt.subplots(3,3, figsize=(7,7))

for i, prog in enumerate(B[:buckets_num+1]):
    filter_ = d_weights_total.loc[d_weights_total.pi == prog]
    new_d = filter_.groupby('pj').agg({'Weight': 'mean'}).reset_index()
    ax = axes.flatten()[i]
    ax.grid(False)
    sns.scatterplot(x='pj', y='Weight', data=filter_, color=mediumblue, ax=ax, marker='+') 
    sns.regplot(x='pj', y='Weight', data=new_d, scatter=True, color=red, ax=ax) 
    ax.set_xlabel('$\eta_k$', fontsize=16) 
    ax.set_ylabel('$\phi^{'+str(round(prog,2))+'}$($\eta_k$)', fontsize=16) 
    print(sum(new_d.Weight))

fig.delaxes(axes[2][1])
fig.delaxes(axes[2][2])
plt.suptitle('$\phi$ function for various expertise levels', fontsize=fontsize_title)
plt.tight_layout()

if focus_on == 'B1': 
    plt.savefig('phi_batch1_gmm.pdf')
else: 
    plt.savefig('phi_7_q.pdf')

In [ ]:
new_d = d_weights_total.groupby('pj').agg({'Weight': 'mean'}).reset_index() 
new_d.Weight = new_d.Weight.divide(sum(new_d.Weight))
sns.scatterplot(x='pj', y='Weight', data=d_weights_total, color=mediumblue, marker='+') 
sns.regplot(x='pj', y='Weight', data=new_d, scatter=True, color=red) 
plt.suptitle('Estimation of $\phi$ Across all Levels', fontsize=fontsize_title) 
plt.xlabel("$\eta_{k}$", fontsize = fontsize_axis)
plt.ylabel("$\phi(\eta_{k})$", fontsize = fontsize_axis) 
plt.tight_layout()
print(sum(new_d.Weight))

if focus_on == 'B1': 
    plt.savefig('phi_agg_pre_gmm.pdf')
else: 
    plt.savefig('phi_agg_7_q.pdf')

In [ ]:
x = np.array(list(new_d.pj)).reshape((-1, 1))
y = np.array(list(new_d.Weight))
model = LinearRegression().fit(x, y)
r_sq = model.score(x, y)
print('coefficient of determination:', r_sq)
print('intercept:', model.intercept_)
print('slope:', model.coef_)
        
x = np.array(list(d_weights_total.pj)).reshape((-1, 1))
y = np.array(list(d_weights_total.Weight))
model = LinearRegression().fit(x, y)
r_sq = model.score(x, y)
print('coefficient of determination:', r_sq)
print('intercept:', model.intercept_)
print('slope:', model.coef_)

In [ ]:
if focus_on == 'B1': 
    d_weights_total.to_csv('phi_w_batch1_k.csv')
else: 
    d_weights_total.to_csv('phi_w_gmm.csv')

In [ ]:
d_weights_total = pd.read_csv('phi_w_gmm.csv')

In [ ]:
res = stats.kendalltau(d_weights_total.Weight, d_weights_total.pj)
print(res)
for name, group in d_weights_total.groupby('pi'):
    print(stats.kendalltau(group.Weight, group.pj))

In [ ]:
sns.lineplot(data=merged_df, x="Labels_GMM", y="ProbaDel", err_style="band", errorbar=("se", 2),)
plt.suptitle('Estimation of $q$ Across all Levels', fontsize=fontsize_title) 
plt.xlabel("$\eta_{k}$", fontsize = fontsize_axis) 
plt.ylabel("$q(\eta_{k})$", fontsize = fontsize_axis)
plt.tight_layout()
plt.savefig('q_agg_b1.pdf')

# Liquid v. Direct

This section compares the performance of liquid and direct democracy

In [ ]:
merged_df_flat['LDest'] = merged_df_flat.Answer*merged_df_flat.Weight
newdd = merged_df_flat[[
         'ID',
         'DelID',
         'GuruID',
         'Answer',
         'AverageExpertise',
         'Weight',
         'Confidence',
         'AvergeConfidence',
         'CodeQuestion',
         'CodeExclu',
         'ExpNum',
         'gender',
         'unique_id', 'IRTExpertise', 'Normalized_IRT','Normalized_IRT_1']] 

newdd['Cat'] = [0 for k in range (len(newdd))]
newld = merged_df_flat[[
         'ID',
         'DelID',
         'GuruID',
         'LDest',
         'AverageExpertise',
         'Weight',
         'Confidence',
         'AvergeConfidence',
         'CodeQuestion',
         'CodeExclu',
         'ExpNum',
         'gender',
         'unique_id', 'IRTExpertise', 'Normalized_IRT','Normalized_IRT_1']] 
newld['Cat'] = [1 for k in range (len(newld))]
newdd = newdd.rename(columns={'Answer':'Estimate'}) 
newld = newld.rename(columns={'LDest':'Estimate'}) 
df_flat = pd.concat([newdd, newld], sort=False) 
np.mean(newld.Estimate), np.mean(newdd.Estimate)

In [ ]:
codes_agg1 = []
codes_agg2 = []
codes_agg3 = []
dd_agg, fd_agg = [], []
s_dd_agg, s_fd_agg = [], []
for name, group in exp_filter.groupby('ExpNum'):
    for name1, group1 in group.groupby('CodeQuestion'):
        dd = np.mean(group1.Answer)
        fd = np.mean(group1.Answer*group1.Weight)
        sdd = np.var(group1.Answer)
        sfd = np.var(group1.Answer*group1.Weight)
        c1, c2, c3 = name, name1, list(group1.CodeExclu)[0]
        dd_agg.append(dd)
        fd_agg.append(fd)
        s_dd_agg.append(dd)
        s_fd_agg.append(fd)
        codes_agg1.append(c1)
        codes_agg2.append(c2)
        codes_agg3.append(c3)

newdd = pd.DataFrame(list(zip(codes_agg1, codes_agg2, codes_agg3, dd_agg)), columns=['ExpNum', 'CodeQuestion', 'CodeExclu', 'DD'])
newfd = pd.DataFrame(list(zip(codes_agg1, codes_agg2, codes_agg3, fd_agg)), columns=['ExpNum', 'CodeQuestion', 'CodeEx clu', 'LD'])
newdd['Cat'] = [0 for k in range (len(newdd))]
newfd['Cat'] = [1 for k in range (len(newfd))]
newdd = newdd.rename(columns={'DD':'Estimate'}) 
newfd = newfd.rename(columns={'LD':'Estimate'}) 

if focus_on == 'B1':
    pd.concat([newdd, newfd], sort=False).to_csv('LDvDD_batch1.csv') 
else:
    pd.concat([newdd, newfd], sort=False).to_csv('LDvDD.csv')

In [ ]:
codes_agg1 = []
codes_agg2 = []
codes_agg3 = []
dd_agg, fd_agg = [], []
s_dd_agg, s_fd_agg = [], []
for name, group in merged_df.groupby('ExpNum'):
    for name1, group1 in merged_df.groupby('CodeExclu'):
        dd = np.mean(group1.Normalized_IRT)
        fd = np.mean(group1.Normalized_IRT*group1.Weight)
        sdd = np.var(group1.Normalized_IRT)
        sfd = np.var(group1.Normalized_IRT*group1.Weight)
        c1, c2 = name, name1
        dd_agg.append(dd)
        fd_agg.append(fd)
        s_dd_agg.append(dd)
        s_fd_agg.append(fd)
        codes_agg1.append(c1)
        codes_agg2.append(c2)

newdd = pd.DataFrame(list(zip(codes_agg1, codes_agg2, dd_agg)), columns=['ExpNum', 'CodeExclu', 'DD'])
newfd = pd.DataFrame(list(zip(codes_agg1, codes_agg2, fd_agg)), columns=['ExpNum', 'CodeExclu', 'LD']) 
newdd['Cat'] = [0 for k in range (len(newdd))]
newfd['Cat'] = [1 for k in range (len(newfd))]
newdd = newdd.rename(columns={'DD':'Estimate'})
newfd = newfd.rename(columns={'LD':'Estimate'}) 

if focus_on == 'B1':
    pd.concat([newdd, newfd], sort=False).to_csv('LDvDD_means_batch1.csv') 
else:
    pd.concat([newdd, newfd], sort=False).to_csv('LDvDD_means.csv')

In [ ]:
# Calculate the average for 'Direct' and 'Liquid' by 'CodeExclu'
both, none, direct, liquid = 0, 0, 0, 0
exp, code, codee, est, outcome_d, outcome_l = [], [], [], [], [], []
for i, r in merged_df_flat.groupby(['ExpNum', 'CodeQuestion']):
    qboth, qnone, qdirect, qliquid = 0, 0, 0, 0
    exp.append(i[0])
    code.append(i[1])
    codee.append(list(r.CodeExclu)[0])
    if sum([np.mean(r.Answer)>0.5, np.mean(r.Answer*r.Weight)>0.5]) == 2:
        both += 1
        qboth += 1
        outcome_d.append(1)
        outcome_l.append(1)
        est.append(1)
        est.append(0)
    if sum([np.mean(r.Answer)>0.5, np.mean(r.Answer*r.Weight)>0.5]) == 0:
        none += 1
        qnone += 1
        outcome_d.append(0)
        outcome_l.append(0)
        est.append(1)
        est.append(0)
    if sum([np.mean(r.Answer)>0.5, np.mean(r.Answer*r.Weight)>0.5]) == 1:
        if np.mean(r.Answer)>0.5:
            direct += 1
            print(i)
            outcome_l.append(0)
            outcome_d.append(1)
            est.append(1)
            est.append(0)
        else:
            liquid +=1 
            print('w')
            print(i)
            outcome_l.append(1)
            outcome_d.append(0)
            est.append(1)
            est.append(0)

correct = pd.DataFrame({'ExpNum':exp, 'CodeQuestion':code, 'CodeExclu': codee, 'Direct':outcome_d, 'Liquid':outcome_l})
grouped_data = correct.groupby('CodeExclu').mean().sort_values('Direct') 

# Reset index for plotting
grouped_data = grouped_data.reset_index() 

# Set up the plot
plt.figure(figsize=(4,5))
plt.scatter(x=grouped_data['CodeExclu'].map(dict_labels), y=grouped_data['Direct'], marker='o', label='Direct', color = mediumblue)
plt.scatter(x=grouped_data['CodeExclu'].map(dict_labels), y=grouped_data['Liquid'], marker='^', label='Liquid', color = red)

plt.xlabel('Task', fontsize=fontsize_axis)
plt.ylabel('Correct Outcome Rate', fontsize=fontsize_axis)
plt.title('Frequency at Which \n Liquid and Direct \n Democracies Are Correct', fontsize=fontsize_title, pad=20) 
plt.legend()
plt.grid(False)
plt.xticks(rotation=90, fontsize=15)
plt.yticks(rotation=0, fontsize=15)
plt.tick_params(which='minor', bottom=False)
plt.legend(fontsize=15, loc='upper left')
plt.tight_layout()

if focus_on == 'B1': 
    plt.savefig('correct_batch1.pdf',bbox_inches='tight')
else: 
    plt.savefig('correct.pdf',bbox_inches='tight')